In [1]:
# %pip install tqdm
# %pip install pillow
# %pip install pandas
# %pip install pytest-plt
# %pip install openpyxl
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
from algorithms.black_hole import BlackHole
from algorithms.simulated_annealing import SimulatedAnnealing
from algorithms.genetic_algorithm import GeneticAlgorithm
from algorithms.lagrangian_heuristics import LagrangianHeuristics
from algorithms.hybrid_approach import GA_SA
from algorithms.hybrid_approach import LH_SA
from algorithms.hybrid_approach import BH_SA
from algorithms.hybrid_approach import LH_BH
from algorithms.hybrid_approach import SA_GA
from services.generation import Test

In [4]:
def start_algorithm(alg, params, name, optimum=None):
    data = []
    for _ in range(5):
        if name[-5:] != 'BH_AD':
            results, times, solution = alg(*params, optimum=optimum)
        else:
            results, times, counts, solution = alg(*params, optimum=optimum)
            plt.plot(counts)
            plt.savefig(f'experiments/charts/{name + '_counts'}.png')
            plt.clf()
        data.append({'results': results, 'time': times})
    if len(data[0]['results']) == 2 and isinstance(data[0]['results'][0], list):
        data = min(data, key=lambda x: min(x['results'][0] + x['results'][1]))
        plt.plot(data['results'][0])
        bound = len(data['results'][0]) - 1
        plt.plot(list(range(bound, bound + len(data['results'][1]))), data['results'][1])
        plt.savefig(f'experiments/charts/{name}.png')
        plt.clf()
        print(min(data['results'][0]), min(data['results'][1]))
        data['results'] = data['results'][0] + data['results'][1]
    else:
        data = min(data, key=lambda x: (x['results'][-1], x['time'][x['results'].index(x['results'][-1])]))
        plt.plot(data['results'])
        plt.savefig(f'experiments/charts/{name}.png')
        plt.clf()
    result = min(data['results'])
    return result,  round(data['time'][data['results'].index(result)], 2)

In [5]:
def start_experiments(algorithms, files, myTest=False):
    frames = []
    for file in files:
        FILE = f'tests/{file['name']}.txt'
        test = Test()
        if myTest:
            test.load_generated_data(FILE)
        else:
            test.load_or_library_data(FILE)
        data = {'OPT': file['optimum']}

        for algorithm in algorithms:
            exp = algorithm['class'](test)
            print(file['name'] + ':', algorithm['name'])
            val, time = start_algorithm(exp.start, algorithm['params'], f'{file['name']}_{algorithm['name']}', 
                                        file['optimum'])
            data[f'{algorithm['name']} value'] = val
            data[f'{algorithm['name']} time'] = time
        print(data)
        frames.append(pd.DataFrame(data, index=[file['name']]))
    df = pd.concat(frames)
    df.to_excel(f'experiments/tables/{'-'.join([alg['name'] for alg in algorithms])}_{int(myTest)}.xlsx')  
    return df


In [1]:
files = [{'name': '4', 'optimum': 429},
         {'name': '5', 'optimum': 253},
         {'name': '6', 'optimum': 138},
         {'name': 'A', 'optimum': 253},
         {'name': 'B', 'optimum': 69},
         {'name': 'C', 'optimum': 227},
         {'name': 'D', 'optimum': 60},
         {'name': 'E', 'optimum': 29},
         {'name': 'F', 'optimum': 14},
         {'name': 'G', 'optimum': 179},
         {'name': 'H', 'optimum': 64}]

algorithms = [{'name': 'BH', 'params': [500, 20, False], 'class': BlackHole},
              {'name': 'BH_AD', 'params': [500, 20, True], 'class': BlackHole}]
start_experiments(algorithms, files)

In [2]:
files = [{'name': '4', 'optimum': 429},
         {'name': '5', 'optimum': 253},
         {'name': '6', 'optimum': 138},]

algorithms = [ {'name': 'GA_ELS', 'params': [20, 0.5, (0, 0, 100), (0, 100, 0, 0), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_PRS', 'params': [20, 0.5, (0, 100, 0), (0, 100, 0, 0), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_RAS', 'params': [20, 0.5, (100, 0, 0), (0, 100, 0, 0), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_RAPRS', 'params': [20, 0.5, (50, 50, 0), (0, 100, 0, 0), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_RAELS', 'params': [20, 0.5, (50, 0, 50), (0, 100, 0, 0), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_PRELS', 'params': [20, 0.5, (0, 50, 50), (0, 100, 0, 0), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_ALLS', 'params': [20, 0.5, (33, 33, 34), (0, 100, 0, 0), (0, 0), 50], 
               'class': GeneticAlgorithm}]
start_experiments(algorithms, files)

In [3]:
files = [{'name': '4', 'optimum': 429},
         {'name': '5', 'optimum': 253},
         {'name': '6', 'optimum': 138},]

algorithms = [ {'name': 'GA_ONC', 'params': [20, 0.5, (0, 0, 100), (0, 100, 0, 0), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_RAC', 'params': [20, 0.5, (0, 0, 100), (100, 0, 0, 0), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_TWC', 'params': [20, 0.5, (0, 0, 100), (0, 0, 100, 0), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_UNC', 'params': [20, 0.5, (0, 0, 100), (0, 0, 0, 100), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_RAONC', 'params': [20, 0.5, (0, 0, 100), (50, 50, 0, 0), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_TWUNC', 'params': [20, 0.5, (0, 0, 100), (0, 0, 50, 50), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_ONUNC', 'params': [20, 0.5, (0, 0, 100), (0, 50, 0, 50), (0, 0), 50], 
               'class': GeneticAlgorithm}, 
               {'name': 'GA_ONTWC', 'params': [20, 0.5, (0, 0, 100), (0, 50, 50, 0), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_RATWC', 'params': [20, 0.5, (0, 0, 100), (50, 0, 50, 0), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_RAUNC', 'params': [20, 0.5, (0, 0, 100), (50, 0, 0, 50), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_ALLC', 'params': [20, 0.5, (0, 0, 100), (25, 25, 25, 25), (0, 0), 50], 
               'class': GeneticAlgorithm}]
start_experiments(algorithms, files)

In [4]:
files1 = [{'name': '4', 'optimum': 429},
         {'name': '5', 'optimum': 253},
         {'name': '6', 'optimum': 138},]

files2= [{'name': 'test1', 'optimum': None},
         {'name': 'test2', 'optimum': None},
         {'name': 'test3', 'optimum': None},
         {'name': 'test4', 'optimum': None}]

algorithms = [
               {'name': 'GA_PRS', 'params': [20, 0.5, (0, 100, 0), (0, 100, 0, 0), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_RAS', 'params': [20, 0.5, (100, 0, 0), (0, 100, 0, 0), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_RAPRS', 'params': [20, 0.5, (50, 50, 0), (0, 100, 0, 0), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_RAELS', 'params': [20, 0.5, (50, 0, 50), (0, 100, 0, 0), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_PRELS', 'params': [20, 0.5, (0, 50, 50), (0, 100, 0, 0), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_ALLS', 'params': [20, 0.5, (33, 33, 34), (0, 100, 0, 0), (0, 0), 50], 
               'class': GeneticAlgorithm}]
start_experiments(algorithms, files2, True)
start_experiments(algorithms, files1)

algorithms = [ {'name': 'GA_ONC', 'params': [20, 0.5, (0, 0, 100), (0, 100, 0, 0), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_RAC', 'params': [20, 0.5, (0, 0, 100), (100, 0, 0, 0), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_TWC', 'params': [20, 0.5, (0, 0, 100), (0, 0, 100, 0), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_UNC', 'params': [20, 0.5, (0, 0, 100), (0, 0, 0, 100), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_RAONC', 'params': [20, 0.5, (0, 0, 100), (50, 50, 0, 0), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_TWUNC', 'params': [20, 0.5, (0, 0, 100), (0, 0, 50, 50), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_ONUNC', 'params': [20, 0.5, (0, 0, 100), (0, 50, 0, 50), (0, 0), 50], 
               'class': GeneticAlgorithm}, 
               {'name': 'GA_ONTWC', 'params': [20, 0.5, (0, 0, 100), (0, 50, 50, 0), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_RATWC', 'params': [20, 0.5, (0, 0, 100), (50, 0, 50, 0), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_RAUNC', 'params': [20, 0.5, (0, 0, 100), (50, 0, 0, 50), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_ALLC', 'params': [20, 0.5, (0, 0, 100), (25, 25, 25, 25), (0, 0), 50], 
               'class': GeneticAlgorithm}]

start_experiments(algorithms, files2, True)

In [6]:
files1 = [{'name': '4', 'optimum': 429},
         {'name': '5', 'optimum': 253},
         {'name': '6', 'optimum': 138},]

files2= [{'name': 'test1', 'optimum': None},
         {'name': 'test2', 'optimum': None},
         {'name': 'test3', 'optimum': None},
         {'name': 'test4', 'optimum': None}]

algorithms = [ {'name': 'GA_0F', 'params': [20, 0, (50, 50, 0), (100, 0, 0, 0), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_25F', 'params': [20, 0.25, (50, 50, 0), (100, 0, 0, 0), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_50F', 'params': [20, 0.5, (50, 50, 0), (100, 0, 0, 0), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_75F', 'params': [20, 0.75, (50, 50, 0), (100, 0, 0, 0), (0, 0), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_100F', 'params': [20, 1, (50, 50, 0), (100, 0, 0, 0), (0, 0), 50], 
               'class': GeneticAlgorithm}]

start_experiments(algorithms, files2, True)
start_experiments(algorithms, files1)

algorithms = [ {'name': 'GA_ELS', 'params': [20, 0.5, (0, 0, 100), (0, 100, 0, 0), (0, 0), 50], 
               'class': GeneticAlgorithm}]
start_experiments(algorithms, files2, True)

In [9]:
files1 = [{'name': '4', 'optimum': 429},
         {'name': '5', 'optimum': 253},
         {'name': '6', 'optimum': 138},]

files2= [{'name': 'test1', 'optimum': None},
         {'name': 'test2', 'optimum': None},
         {'name': 'test3', 'optimum': None},
         {'name': 'test4', 'optimum': None}]

algorithms = [ {'name': 'GA_1000-25PS', 'params': [20, 0, (50, 50, 0), (100, 0, 0, 0), (1000, 25), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_1000-50PS', 'params': [20, 0, (50, 50, 0), (100, 0, 0, 0), (1000, 50), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_1000-75PS', 'params': [20, 0, (50, 50, 0), (100, 0, 0, 0), (1000, 75), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_500-25PS', 'params': [20, 0, (50, 50, 0), (100, 0, 0, 0), (500, 25), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_500-50PS', 'params': [20, 0, (50, 50, 0), (100, 0, 0, 0), (500, 50), 50], 
               'class': GeneticAlgorithm},
               {'name': 'GA_500-75PS', 'params': [20, 0, (50, 50, 0), (100, 0, 0, 0), (500, 75), 50], 
               'class': GeneticAlgorithm}]

start_experiments(algorithms, files2, True)
start_experiments(algorithms, files1)

In [10]:
algorithms = [{'name': 'GA_BEST', 'params': [20, 0, (50, 50, 0), (100, 0, 0, 0), (0, 0), 500], 
               'class': GeneticAlgorithm}]

files1 = [{'name': '4', 'optimum': 429},
         {'name': '5', 'optimum': 253},
         {'name': '6', 'optimum': 138},
         {'name': 'A', 'optimum': 253},
         {'name': 'B', 'optimum': 69},
         {'name': 'C', 'optimum': 227},
         {'name': 'D', 'optimum': 60},
         {'name': 'E', 'optimum': 29},
         {'name': 'F', 'optimum': 14},
         {'name': 'G', 'optimum': 179},
         {'name': 'H', 'optimum': 64}]

files2= [{'name': 'test1', 'optimum': None},
         {'name': 'test2', 'optimum': None},
         {'name': 'test3', 'optimum': None},
         {'name': 'test4', 'optimum': None}]

start_experiments(algorithms, files2, True)
start_experiments(algorithms, files1)

In [11]:
algorithms = [{'name': 'BH_AD', 'params': [500, 20, True], 'class': BlackHole}]
files= [{'name': 'test1', 'optimum': 899},
         {'name': 'test2', 'optimum': 371},
         {'name': 'test3', 'optimum': 1435},
         {'name': 'test4', 'optimum': 664}]
start_experiments(algorithms, files, True)

In [12]:
algorithms = [{'name': 'LH', 'params': [5000], 'class': LagrangianHeuristics}]
files1 = [{'name': 'test1', 'optimum': 899},
         {'name': 'test2', 'optimum': 371},
         {'name': 'test3', 'optimum': 1435},
         {'name': 'test4', 'optimum': 664}
         ]

files2 = [{'name': '4', 'optimum': 429},
         {'name': '5', 'optimum': 253},
         {'name': '6', 'optimum': 138},
         {'name': 'A', 'optimum': 253},
         {'name': 'B', 'optimum': 69},
         {'name': 'C', 'optimum': 227},
         {'name': 'D', 'optimum': 60},
         {'name': 'E', 'optimum': 29},
         {'name': 'F', 'optimum': 14},
         {'name': 'G', 'optimum': 179},
         {'name': 'H', 'optimum': 64}]

start_experiments(algorithms, files1, True)
start_experiments(algorithms, files2)

In [13]:
algorithms = [{'name': 'SA', 'params': [500, 10000], 'class': SimulatedAnnealing}, ]
files1 = [{'name': 'test1', 'optimum': 899},
         {'name': 'test2', 'optimum': 371},
         {'name': 'test3', 'optimum': 1435},
         {'name': 'test4', 'optimum': 664}]

files2 = [{'name': '4', 'optimum': 429},
         {'name': '5', 'optimum': 253},
         {'name': '6', 'optimum': 138},
         {'name': 'A', 'optimum': 253},
         {'name': 'B', 'optimum': 69},
         {'name': 'C', 'optimum': 227},
         {'name': 'D', 'optimum': 60},
         {'name': 'E', 'optimum': 29},
         {'name': 'F', 'optimum': 14},
         {'name': 'G', 'optimum': 179},
         {'name': 'H', 'optimum': 64}]

start_experiments(algorithms, files1, True)
start_experiments(algorithms, files2)

In [14]:
algorithms = [{'name': 'GA_SA', 'params': [20, 0, (50, 50, 0), (100, 0, 0, 0), (0, 0), 250, 500, 5000], 
               'class': GA_SA}, ]

files = [{'name': '4', 'optimum': 429},
         {'name': '5', 'optimum': 253},
         {'name': '6', 'optimum': 138},
         {'name': 'A', 'optimum': 253},
         {'name': 'C', 'optimum': 227},
         {'name': 'D', 'optimum': 60},
         {'name': 'F', 'optimum': 14},
         {'name': 'G', 'optimum': 179},
         {'name': 'H', 'optimum': 64}
         ]

start_experiments(algorithms, files)

In [15]:
algorithms = [{'name': 'LH_SA', 'params': [2500, 500, 10000], 
               'class': LH_SA}, ]

files = [{'name': '4', 'optimum': 429},
         {'name': '5', 'optimum': 253},
         {'name': '6', 'optimum': 138},
         {'name': 'A', 'optimum': 253},
         {'name': 'B', 'optimum': 69},
         {'name': 'C', 'optimum': 227},
         {'name': 'D', 'optimum': 60},
         {'name': 'F', 'optimum': 14},
         {'name': 'G', 'optimum': 179},
         {'name': 'H', 'optimum': 64}
         ]

start_experiments(algorithms, files)

In [16]:
algorithms = [{'name': 'BH_SA', 'params': [500, 20, 500, 5000], 
               'class': BH_SA}, ]

files = [{'name': '4', 'optimum': 429},
         {'name': '5', 'optimum': 253},
         {'name': 'A', 'optimum': 253},
         {'name': 'C', 'optimum': 227},
         {'name': 'G', 'optimum': 179},
         {'name': 'H', 'optimum': 64}
         ]

start_experiments(algorithms, files)

In [17]:
algorithms = [{'name': 'LH_BH', 'params': [5000, 500, 20], 
               'class': LH_BH}, ]

files = [{'name': '4', 'optimum': 429},
         {'name': '5', 'optimum': 253},
         {'name': 'A', 'optimum': 253},
         {'name': 'C', 'optimum': 227},
         {'name': 'D', 'optimum': 60},
         {'name': 'G', 'optimum': 179},
         {'name': 'H', 'optimum': 64}
         ]

start_experiments(algorithms, files)

In [18]:
algorithms = [{'name': 'SA_GA', 'params': [500, 5000, 20, 0, (50, 50, 0), (100, 0, 0, 0), (0, 0), 250], 
               'class': SA_GA}, ]

files = [{'name': '4', 'optimum': 429},
         {'name': '5', 'optimum': 253},
         {'name': '6', 'optimum': 138},
         {'name': 'A', 'optimum': 253},
         {'name': 'C', 'optimum': 227},
         {'name': 'D', 'optimum': 60},
         {'name': 'F', 'optimum': 14},
         {'name': 'G', 'optimum': 179},
         {'name': 'H', 'optimum': 64}
         ]

start_experiments(algorithms, files)